# 🚀 Apex-X Training on A100 SXM (80GB)

This notebook is optimized for training Apex-X on large-scale datasets using **NVIDIA A100 80GB**. 

### 🏗️ Hardware Specs:
- **GPU**: 1x A100 SXM (80 GB VRAM)
- **RAM**: 117 GB
- **CPU**: 16 vCPU

### 🎯 Dataset:
- **Path**: `/media/voskan/New Volume/2TB HDD/YOLO26_SUPER_MERGED`
- **Format**: YOLO Segmentation

## 1. Setup Environment

In [ ]:
# Clone repository if not already present
import os
if not os.path.exists('Apex-X'):
    !git clone https://github.com/Voskan/Apex-X.git

%cd Apex-X

# Install dependencies
!pip install -e .
!pip install pycocotools albumentations wandb

## 2. Configuration

Optimized for 80GB VRAM. We can use a large batch size and high-resolution images.

In [ ]:
import torch
from apex_x.data import YOLOSegmentationDataset, build_robust_transforms, MosaicAugmentation, MixUpAugmentation
from apex_x.model import TeacherModel
from apex_x.config import ApexXConfig
from apex_x.train import ApexXTrainer, LinearWarmupCosineAnnealingLR
from torch.utils.data import DataLoader
from pathlib import Path

# --- Hyperparameters ---
DATASET_ROOT = "/media/voskan/New Volume/2TB HDD/YOLO26_SUPER_MERGED"
IMAGE_SIZE = 1024  # High res for satellite imagery
BATCH_SIZE = 64    # Optimized for A100 80GB
EPOCHS = 300
BASE_LR = 0.01
WEIGHT_DECAY = 0.0005
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUTPUT_DIR = "./outputs/a100_training"

print(f"Device: {DEVICE}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 3. Data Loading

Using the native `YOLOSegmentationDataset` with Mosaic and MixUp augmentations.

In [ ]:
# Build Base Dataset
train_dataset = YOLOSegmentationDataset(
    root=DATASET_ROOT, 
    split="train",
    image_size=IMAGE_SIZE
)

# Apply Advanced Augmentations
print("Enabling Mosaic & MixUp...")
train_dataset.transforms = MosaicAugmentation(
    dataset=train_dataset,
    output_size=IMAGE_SIZE,
    mosaic_prob=0.5
)

# Standard Albumentations Pipeline
standard_transforms = build_robust_transforms(image_size=IMAGE_SIZE, is_training=True)

# Patch-based training (Optional but recommended for memory efficiency if increasing resolution further)
# For 1024px on A100, we can stay full-frame or use patches.

val_dataset = YOLOSegmentationDataset(
    root=DATASET_ROOT,
    split="val",
    image_size=IMAGE_SIZE,
    transforms=build_robust_transforms(image_size=IMAGE_SIZE, is_training=False)
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=16, # Optimized for 16 vCPUs
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
    pin_memory=True
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

## 4. Model & Trainer

In [ ]:
config = ApexXConfig(
    num_classes=train_dataset.num_classes,
    image_size=IMAGE_SIZE
)

model = TeacherModel(config).to(DEVICE)

trainer = ApexXTrainer(
    teacher=model,
    device=DEVICE,
    checkpoint_dir=Path(OUTPUT_DIR) / "checkpoints"
)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=BASE_LR,
    weight_decay=WEIGHT_DECAY
)

scheduler = LinearWarmupCosineAnnealingLR(
    optimizer=optimizer,
    warmup_epochs=10,
    total_epochs=EPOCHS,
    eta_min=BASE_LR * 0.01
)

# Mixed Precision Training (Automatic)
scaler = torch.cuda.amp.GradScaler() if DEVICE == "cuda" else None

## 5. Training Loop

In [ ]:
import wandb
wandb.init(project="Apex-X-A100-SXM", config={
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "lr": BASE_LR,
    "image_size": IMAGE_SIZE
})

best_map = 0.0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    # Train phase
    train_metrics = trainer.train_epoch(
        train_loader,
        optimizer,
        epoch=epoch,
        total_epochs=EPOCHS,
        scaler=scaler  # Enables FP16
    )
    
    scheduler.step()
    
    # Validate
    if (epoch + 1) % 5 == 0:
        val_metrics = trainer.validate(val_loader, compute_map=True)
        current_map = val_metrics.get("mAP_bbox", 0.0)
        
        if current_map > best_map:
            best_map = current_map
            trainer.save_checkpoint(
                Path(OUTPUT_DIR) / "checkpoints" / "best.pt",
                epoch=epoch,
                metrics=val_metrics
            )
        
        wandb.log({"val/mAP": current_map, "val/loss": val_metrics.get("val_loss")})
    
    wandb.log(train_metrics)

wandb.finish()